# Step 2 - Extract image labels with Image Classification

This script is to extract image labels as keywords that will also be embedded into sentence vectors. The purpose is to increase searchability by including visual features as keywords of an image. For example, in an ideal scenario, if an image with a backpack is correctly classified, we will get a keyword "backpack" and embed this word into the sentence vecotr. When given an inquiry of "backpack" or similar words "bag", the retrieval results could include this image as it contains semantically similar words.

Prerequisites:
    Keras, numpy
    
Input: 
    1. database file
    
Output:
    1. new database file

In [1]:
%pylab inline
import os
import json
from imageextractor import ImageClassifier


Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [ ]:
"""
Make sure these variables are correctly set.
model: model name inceptionv3|vgg19
input_shape: different model requires different image input shape. 
            The required input image has to be in shape(299,299) for InceptionV3 and (224, 224) for VGG19
database_file: file path to database file generated from previous steps
output_file: a new database file with image keywords
top_n_keywords: specify top n keywords to be extracted
"""

model='inceptionv3'
input_shape = (299, 299)
database_file = './data source/Life Is Strange 1/output/Life Is Strange.json'
output_file = './data source/Life Is Strange 1/output/lifeisstrange_w_img_top3keywords.json'
top_n_keywords = 3


In [16]:
%%time
extractor = ImageClassifier(model=model, resize_shape=input_shape)

Wall time: 16.5 s


In [19]:
%%time

json_file = open(database_file)
database = json.loads(json_file.read())
for file_id, file_info in database.items():
    folder = file_info['screenshots']['image_folder']
    print(folder)
    sessions = file_info['screenshots']['image_info']
    for session_id, session in sessions.items():
        for key, image in session['frames'].items():
            
            file_path = os.path.join(folder, image['image_file'])
            img = extractor.preprocess(file_path)
            labels = extractor.extract_labels(img, top=top_n_keywords)
            additional_texts = ' '.join(labels)
            session['frames'][key]['image_keywords'] = additional_texts

with open(output_file, 'w+') as out:
    json.dump(database, out, indent=4)

./data source/Life Is Strange 1/output/Episode_1_Chrysalis
./data source/Life Is Strange 1/output/Episode_2_Out_of_Time
./data source/Life Is Strange 1/output/Episode_3_Chaos_Theory
./data source/Life Is Strange 1/output/Episode_4_Dark_Room
./data source/Life Is Strange 1/output/Episode_5_Polarized
Wall time: 1h 40min 10s


In [ ]:
# # Test
# image_file = 'Frame_00112333.jpg'
# img = extractor.preprocess(image_file)
# labels = extractor.extract_labels(img, top=3)
# print(labels)